In [1]:
import os, numpy as np, onnx, onnxruntime as ort
from pathlib import Path

print('onnxruntime', ort.__version__)

onnxruntime 1.19.2


In [7]:
# 🔧 ---------------- Configuration ----------------
MODEL_PATH = Path('./RealTimeAlignment/onnx_no-residual/submodule_onnx/submodule_embed.onnx')   # update as necessary
DUMP_DIR   = Path('embed_dumps')
DUMP_DIR.mkdir(exist_ok=True)

# Optional: external input file (.npy or .txt); leave None to use random data
INPUT_FILE = None
BATCH_SIZE = 1
# -------------------------------------------------

In [8]:
# Prepare input
def load_input():
    if INPUT_FILE is None:
        print('[INFO] No INPUT_FILE provided – generating random input')
        x = np.random.randn(BATCH_SIZE, 6).astype(np.float32)
    else:
        if INPUT_FILE.suffix == '.npy':
            x = np.load(INPUT_FILE).astype(np.float32)
        else:
            x = np.loadtxt(INPUT_FILE, dtype=np.float32)
        x = x.reshape(BATCH_SIZE, 6).astype(np.float32)
    np.savetxt(DUMP_DIR/'input_dense1.txt', x.flatten(), fmt='%.6f')
    return x

x_in = load_input()
print('Input shape:', x_in.shape)

[INFO] No INPUT_FILE provided – generating random input
Input shape: (1, 6)


In [9]:
# Dump weights/biases
model = onnx.load(str(MODEL_PATH))
initializer_map = {init.name: init for init in model.graph.initializer}

def tensor_to_numpy(tensor):
    return np.frombuffer(tensor.raw_data,
                         dtype=onnx.mapping.TENSOR_TYPE_TO_NP_TYPE[tensor.data_type]
                         ).reshape(tensor.dims)

def dump_tensor(name, arr):
    np.savetxt(DUMP_DIR/f'{name}.txt', arr.flatten(), fmt='%.6f')

for name, tensor in initializer_map.items():
    arr = tensor_to_numpy(tensor)
    dump_tensor(f'weights_{name}', arr)
    print(f'Dumped {name}  shape={arr.shape}')

Dumped 1.bias  shape=(128,)
Dumped 4.bias  shape=(128,)
Dumped onnx::MatMul_13  shape=(6, 128)
Dumped onnx::MatMul_14  shape=(128, 128)


/tmp/ipykernel_1210996/1062612749.py:7: DeprecationWarning: `mapping.TENSOR_TYPE_TO_NP_TYPE` is now deprecated and will be removed in a future release.To silence this warning, please use `helper.tensor_dtype_to_np_dtype` instead.
  dtype=onnx.mapping.TENSOR_TYPE_TO_NP_TYPE[tensor.data_type]


In [13]:
# Inference with intermediate tensors
sess_opts = ort.SessionOptions()
sess_opts.graph_optimization_level = ort.GraphOptimizationLevel.ORT_DISABLE_ALL
session = ort.InferenceSession(str(MODEL_PATH), sess_opts, providers=['CPUExecutionProvider'])

wanted_ops = {'Gemm', 'LeakyRelu'}
intermediate = []
for node in model.graph.node:
    if node.op_type in wanted_ops:
        intermediate.extend(node.output)

input_name = session.get_inputs()[0].name
results = session.run(intermediate + [session.get_outputs()[0].name],
                      {input_name: x_in})

# for name, arr in zip(intermediate + [session.get_outputs()[0].name], results):
#     dump_tensor(name, arr)
#     print(f'Dumped {name}  shape={arr.shape}')

InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Unexpected input data type. Actual: (tensor(float)) , expected: (tensor(float16))